In [1]:

import pandas as pd
import requests
from io import StringIO


url = f"https://stockanalysis.com/ipos/withdrawn/"
headers = {
    'User-Agent': (
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
        'AppleWebKit/537.36 (KHTML, like Gecko) '
        'Chrome/58.0.3029.110 Safari/537.3'
    )
}

try:
    response = requests.get(url, headers=headers, timeout=10)
    response.raise_for_status()

    # Wrap HTML text in StringIO to avoid deprecation warning
    # "Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object."
    html_io = StringIO(response.text)
    table = pd.read_html(html_io)

    if not table:
        raise ValueError(f"No table found")

    
except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")
except ValueError as ve:
    print(f"Data error: {ve}")
except Exception as ex:
    print(f"Unexpected error: {ex}")

print(type(table))
withdrawn = pd.DataFrame(table[0])
print(len(withdrawn))
pd.set_option('display.max_rows', None)
withdrawn.head(99)

<class 'list'>
99


,Symbol,Company Name,Price Range,Shares Offered
0,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000
1,AURN,"Aurion Biotech, Inc.",-,-
2,ROTR,"PHI Group, Inc.",-,-
3,ONE,One Power Company,-,-
4,HPOT,The Great Restaurant Development Holdings Limited,$4.00 - $6.00,1400000
5,CABR,"Caring Brands, Inc.",$4.00,750000
6,SQVI,"Sequoia Vaccines, Inc.",$8.00 - $10.00,2775000
7,SNI,Shenni Holdings Limited,$4.00 - $6.00,3000000
8,KMCM,Key Mining Corp.,$2.25,4444444
9,XGEN,GenEmbryomics Limited,$4.75,1095000


In [2]:

withdrawn = withdrawn.astype({"Company Name": "string"})
print(withdrawn.dtypes)

Symbol                    object
Company Name      string[python]
Price Range               object
Shares Offered            object
dtype: object


In [2]:
#print(withdrawn['Company Name'])
import re
def company_class(name):
    cc = None
    namel = re.split(r"[, ]+", name)
    if "Acquisition" in namel: cc = "Acq.Corp"
    elif "Group" in namel: cc = "Group"
    elif "Holdings" in namel: cc = "Holdings"
    elif "Ltd" in namel or "Limited" in namel: cc = "Ltd"
    elif "Inc." in namel or "Incorporated" in namel: cc = "Inc"
    else: cc = "Other"
  #  print(namel)
    return cc

ccs = withdrawn['Company Name'].apply(company_class)
withdrawn.insert(2, 'Company Class', ccs)
withdrawn.head()

,Symbol,Company Name,Company Class,Price Range,Shares Offered
0,UNFL,"Unifoil Holdings, Inc.",Holdings,$3.00 - $4.00,2000000
1,AURN,"Aurion Biotech, Inc.",Inc,-,-
2,ROTR,"PHI Group, Inc.",Group,-,-
3,ONE,One Power Company,Other,-,-
4,HPOT,The Great Restaurant Development Holdings Limited,Holdings,$4.00 - $6.00,1400000


In [3]:
print(withdrawn['Company Class'])

0     Holdings
1          Inc
2        Group
3        Other
4     Holdings
5          Inc
6          Inc
7     Holdings
8        Other
9          Ltd
10       Group
11         Inc
12         Inc
13       Group
14         Inc
15         Inc
16         Inc
17       Other
18         Inc
19       Other
20         Inc
21         Inc
22       Group
23         Inc
24         Inc
25         Inc
26    Holdings
27       Other
28    Holdings
29         Inc
30         Inc
31         Inc
32       Group
33         Inc
34    Holdings
35       Other
36         Ltd
37       Group
38         Inc
39         Inc
40         Ltd
41         Inc
42         Inc
43         Inc
44    Acq.Corp
45         Inc
46         Inc
47    Acq.Corp
48         Ltd
49       Group
50         Inc
51       Group
52       Other
53         Ltd
54    Acq.Corp
55         Inc
56       Group
57    Holdings
58         Inc
59    Acq.Corp
60    Acq.Corp
61       Other
62         Ltd
63         Inc
64         Inc
65         Inc
66    Acq.

In [3]:
#print(withdrawn['Price Range'])
def avg_price(price_range):
    prices = re.split(r"[- $]+", price_range)
    prices = [x for x in prices if x != '']
   # print(prices)
    prices = [float(x) for x in prices]
    if not prices: return None
    else: return round(sum(prices)/len(prices), 1)

avgp = withdrawn['Price Range'].apply(avg_price)
withdrawn.insert(4, 'Avg. price', avgp)
withdrawn.head(99)       

,Symbol,Company Name,Company Class,Price Range,Avg. price,Shares Offered
0,UNFL,"Unifoil Holdings, Inc.",Holdings,$3.00 - $4.00,3.5,2000000
1,AURN,"Aurion Biotech, Inc.",Inc,-,NaN,-
2,ROTR,"PHI Group, Inc.",Group,-,NaN,-
3,ONE,One Power Company,Other,-,NaN,-
4,HPOT,The Great Restaurant Development Holdings Limited,Holdings,$4.00 - $6.00,5.0,1400000
5,CABR,"Caring Brands, Inc.",Inc,$4.00,4.0,750000
6,SQVI,"Sequoia Vaccines, Inc.",Inc,$8.00 - $10.00,9.0,2775000
7,SNI,Shenni Holdings Limited,Holdings,$4.00 - $6.00,5.0,3000000
8,KMCM,Key Mining Corp.,Other,$2.25,2.2,4444444
9,XGEN,GenEmbryomics Limited,Ltd,$4.75,4.8,1095000


In [4]:
withdrawn['Shares Offered'] = pd.to_numeric(withdrawn['Shares Offered'], errors='coerce')

withdrawn.head()

,Symbol,Company Name,Company Class,Price Range,Avg. price,Shares Offered
0,UNFL,"Unifoil Holdings, Inc.",Holdings,$3.00 - $4.00,3.5,2000000.0
1,AURN,"Aurion Biotech, Inc.",Inc,-,NaN,NaN
2,ROTR,"PHI Group, Inc.",Group,-,NaN,NaN
3,ONE,One Power Company,Other,-,NaN,NaN
4,HPOT,The Great Restaurant Development Holdings Limited,Holdings,$4.00 - $6.00,5.0,1400000.0


In [5]:
withdrawn = withdrawn.dropna()
withdrawn.head()

,Symbol,Company Name,Company Class,Price Range,Avg. price,Shares Offered
0,UNFL,"Unifoil Holdings, Inc.",Holdings,$3.00 - $4.00,3.5,2000000.0
4,HPOT,The Great Restaurant Development Holdings Limited,Holdings,$4.00 - $6.00,5.0,1400000.0
5,CABR,"Caring Brands, Inc.",Inc,$4.00,4.0,750000.0
6,SQVI,"Sequoia Vaccines, Inc.",Inc,$8.00 - $10.00,9.0,2775000.0
7,SNI,Shenni Holdings Limited,Holdings,$4.00 - $6.00,5.0,3000000.0


In [9]:
print(withdrawn.columns)
withdrawn['Withdrawn Value'] = withdrawn['Shares Offered'] * withdrawn['Avg. price']
print(len(withdrawn))
withdrawn.head()

Index(['Symbol', 'Company Name', 'Company Class', 'Price Range', 'Avg. price',
       'Shares Offered'],
      dtype='object')
71


,Symbol,Company Name,Company Class,Price Range,Avg. price,Shares Offered,Withdrawn Value
0,UNFL,"Unifoil Holdings, Inc.",Holdings,$3.00 - $4.00,3.5,2000000.0,7000000.0
4,HPOT,The Great Restaurant Development Holdings Limited,Holdings,$4.00 - $6.00,5.0,1400000.0,7000000.0
5,CABR,"Caring Brands, Inc.",Inc,$4.00,4.0,750000.0,3000000.0
6,SQVI,"Sequoia Vaccines, Inc.",Inc,$8.00 - $10.00,9.0,2775000.0,24975000.0
7,SNI,Shenni Holdings Limited,Holdings,$4.00 - $6.00,5.0,3000000.0,15000000.0


In [10]:
total_values = withdrawn.groupby('Company Class')['Withdrawn Value'].sum()
print(total_values)

Company Class
Acq.Corp    4.021000e+09
Group       3.664789e+08
Holdings    3.150000e+08
Inc         1.912145e+09
Ltd         2.756560e+08
Other       8.138978e+08
Name: Withdrawn Value, dtype: float64


In [11]:
print(total_values.max())

4021000000.0


In [12]:
print("The class acquision corporation had the highest withdrawn value")

The class acquision corporation had the highest withdrawn value


In [1]:
import pandas as pd
import numpy as np
import requests
from io import StringIO


url = f"https://stockanalysis.com/ipos/2024/"
headers = {
    'User-Agent': (
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
        'AppleWebKit/537.36 (KHTML, like Gecko) '
        'Chrome/58.0.3029.110 Safari/537.3'
    )
}

try:
    response = requests.get(url, headers=headers, timeout=10)
    response.raise_for_status()

    # Wrap HTML text in StringIO to avoid deprecation warning
    # "Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object."
    html_io = StringIO(response.text)
    table = pd.read_html(html_io)

    if not table:
        raise ValueError(f"No table found")

    
except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")
except ValueError as ve:
    print(f"Data error: {ve}")
except Exception as ex:
    print(f"Unexpected error: {ex}")

print(type(table))
ipos2024 = pd.DataFrame(table[0])
ipos2024['IPO Date'] = pd.to_datetime(ipos2024['IPO Date'],  format='%b %d, %Y')
ipos2024 = ipos2024[(ipos2024['IPO Date'] < '2024-06-01') & (ipos2024['IPO Price'].astype(str).str.find('-') == -1)]
print(len(ipos2024))
pd.set_option('display.max_rows', None)
ipos2024.head(77)

<class 'list'>
75


,IPO Date,Symbol,Company Name,IPO Price,Current,Return
149,2024-05-23,BOW,Bowhead Specialty Holdings Inc.,$17.00,$36.84,116.71%
150,2024-05-17,HDL,Super Hi International Holding Ltd.,$19.56,$19.60,0.21%
151,2024-05-17,RFAI,RF Acquisition Corp II,$10.00,$10.52,5.20%
152,2024-05-15,JDZG,JIADE Limited,$4.00,$0.30,-92.64%
153,2024-05-15,RAY,Raytech Holding Limited,$4.00,$1.21,-69.75%
154,2024-05-14,BTOC,Armlogi Holding Corp.,$5.00,$1.51,-69.80%
155,2024-05-10,ZK,ZEEKR Intelligent Technology Holding Limited,$21.00,$26.15,24.52%
156,2024-05-09,GPAT,GP-Act III Acquisition Corp.,$10.00,$10.60,6.00%
157,2024-05-09,PAL,"Proficient Auto Logistics, Inc.",$15.00,$7.01,-53.30%
158,2024-05-09,SVCO,"Silvaco Group, Inc.",$19.00,$4.74,-75.05%


In [2]:
tickers = ipos2024.reset_index()['Symbol'].values.tolist()
print(tickers)
print(type(tickers))

['BOW', 'HDL', 'RFAI', 'JDZG', 'RAY', 'BTOC', 'ZK', 'GPAT', 'PAL', 'SVCO', 'NNE', 'CCIX', 'VIK', 'ZONE', 'LOAR', 'MRX', 'RBRK', 'NCI', 'MFI', 'YYGH', 'TRSG', 'CDTG', 'CTRI', 'IBTA', 'MTEN', 'TWG', 'ULS', 'PACS', 'MNDR', 'CTNM', 'MAMO', 'ZBAO', 'BOLD', 'MMA', 'UBXG', 'IBAC', 'AUNA', 'BKHA', 'LOBO', 'RDDT', 'ALAB', 'INTJ', 'RYDE', 'LGCL', 'SMXT', 'VHAI', 'DYCQ', 'CHRO', 'UMAC', 'HLXB', 'MGX', 'TBBB', 'TELO', 'KYTX', 'PMNT', 'AHR', 'LEGT', 'ANRO', 'GUTS', 'AS', 'FBLG', 'AVBP', 'BTSG', 'HAO', 'CGON', 'YIBO', 'JL', 'SUGP', 'JVSA', 'KSPI', 'CCTG', 'PSBD', 'SYNX', 'SDHC', 'ROMA']
<class 'list'>


In [3]:
import yfinance as yf
import time
stocks_df = pd.DataFrame({'A' : []})

for i, t in enumerate(tickers): 
    print(i, t)
    historyPrices = yf.Ticker(t).history(period = "max", interval = "1d")
    historyPrices['Ticker'] = t
    historyPrices['Year']= historyPrices.index.year
    historyPrices['Month'] = historyPrices.index.month
    historyPrices['Weekday'] = historyPrices.index.weekday
    historyPrices['Date'] = historyPrices.index.date
     # historical returns
    for i in [1,3,7,30,90,252]:
         historyPrices['growth_'+str(i)+'d'] = historyPrices['Close'] / historyPrices['Close'].shift(i)
    # historyPrices['growth_future_30d'] = historyPrices['Close'].shift(-30) / historyPrices['Close']
    
    # # Technical indicators
    # # SimpleMovingAverage 10 days and 20 days
    # historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
    # historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
    # historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
    # historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Close']
    
    # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
    historyPrices['volatility'] =   historyPrices['Close'].rolling(30).std() * np.sqrt(252)
    # what we want to predict
    #historyPrices['is_positive_growth_30d_future'] = np.where(historyPrices['growth_future_30d'] > 1, 1, 0)
    
    # sleep 1 sec between downloads - not to overload the API server
    time.sleep(1)
    
    
    if stocks_df.empty:
        stocks_df = historyPrices
    else:
        stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

0 BOW
1 HDL
2 RFAI
3 JDZG
4 RAY
5 BTOC
6 ZK
7 GPAT
8 PAL
9 SVCO
10 NNE
11 CCIX
12 VIK
13 ZONE
14 LOAR
15 MRX
16 RBRK
17 NCI
18 MFI
19 YYGH
20 TRSG
21 CDTG
22 CTRI
23 IBTA
24 MTEN
25 TWG
26 ULS
27 PACS
28 MNDR
29 CTNM
30 MAMO
31 ZBAO
32 BOLD
33 MMA
34 UBXG
35 IBAC
36 AUNA
37 BKHA
38 LOBO
39 RDDT
40 ALAB
41 INTJ
42 RYDE
43 LGCL
44 SMXT
45 VHAI
46 DYCQ
47 CHRO
48 UMAC
49 HLXB
50 MGX
51 TBBB
52 TELO
53 KYTX
54 PMNT
55 AHR
56 LEGT
57 ANRO
58 GUTS
59 AS
60 FBLG
61 AVBP
62 BTSG
63 HAO
64 CGON
65 YIBO
66 JL
67 SUGP
68 JVSA
69 KSPI
70 CCTG
71 PSBD
72 SYNX
73 SDHC
74 ROMA


In [4]:
pd.set_option('display.max_columns', None)
stocks_df.head(50)

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Year,Month,Weekday,Date,growth_1d,growth_3d,growth_7d,growth_30d,growth_90d,growth_252d,volatility
0,23.000000,24.270000,22.139999,23.799999,3335800,0.0,0.0,BOW,2024,5,3,2024-05-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,24.260000,26.150000,23.980000,25.700001,990500,0.0,0.0,BOW,2024,5,4,2024-05-24,1.079832,NaN,NaN,NaN,NaN,NaN,NaN
2,25.850000,26.879999,25.075001,26.480000,555100,0.0,0.0,BOW,2024,5,1,2024-05-28,1.030350,NaN,NaN,NaN,NaN,NaN,NaN
3,26.440001,26.490000,25.500999,26.290001,302700,0.0,0.0,BOW,2024,5,2,2024-05-29,0.992825,1.104622,NaN,NaN,NaN,NaN,NaN
4,27.209999,27.209999,25.500000,26.139999,200900,0.0,0.0,BOW,2024,5,3,2024-05-30,0.994294,1.017121,NaN,NaN,NaN,NaN,NaN
5,26.490000,26.990000,25.100000,26.799999,198800,0.0,0.0,BOW,2024,5,4,2024-05-31,1.025249,1.012085,NaN,NaN,NaN,NaN,NaN
6,27.000000,27.490000,26.360001,26.639999,283500,0.0,0.0,BOW,2024,6,0,2024-06-03,0.994030,1.013313,NaN,NaN,NaN,NaN,NaN
7,26.129999,26.900000,25.200001,25.230000,169800,0.0,0.0,BOW,2024,6,1,2024-06-04,0.947072,0.965187,1.060084,NaN,NaN,NaN,NaN
8,25.160000,25.900000,24.370001,25.360001,348400,0.0,0.0,BOW,2024,6,2,2024-06-05,1.005153,0.946269,0.986770,NaN,NaN,NaN,NaN
9,25.400000,26.200001,25.170000,25.420000,100000,0.0,0.0,BOW,2024,6,3,2024-06-06,1.002366,0.954204,0.959970,NaN,NaN,NaN,NaN


In [5]:
stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 0.045) / stocks_df['volatility']

In [14]:
print(stocks_df['Date'].dtype)


object


In [6]:
filtered_stocks_df = stocks_df[pd.to_datetime(stocks_df['Date'], format='%Y-%m-%d')  == '2025-06-06']
print(filtered_stocks_df)

             Open        High         Low       Close    Volume  Dividends  \
259     36.439999   36.540001   35.930000   36.389999    137900        0.0   
529     20.430000   20.440001   19.969999   20.410000      1300        0.0   
767     10.510000   10.510000   10.510000   10.510000         0        0.0   
1039     0.380000    0.386000    0.201000    0.295000   3428900        0.0   
1311     1.110000    1.270000    1.100000    1.255000    501000        0.0   
1584     1.210000    1.320000    1.161000    1.290000     21500        0.0   
1859    26.650000   26.650000   25.924999   26.389999    580300        0.0   
2099    10.560000   10.560000   10.560000   10.560000      1800        0.0   
2375     7.710000    7.780000    7.250000    7.320000    123700        0.0   
2651     4.820000    4.959000    4.750000    4.810000     50700        0.0   
2928    29.150000   31.559000   28.350000   31.190001   3244100        0.0   
3175    10.620000   10.700000   10.600000   10.670000    617800 

In [7]:

filtered_stocks_df.head(75)

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Year,Month,Weekday,Date,growth_1d,growth_3d,growth_7d,growth_30d,growth_90d,growth_252d,volatility,Sharpe
259,36.439999,36.540001,35.930000,36.389999,137900,0.0,0.0,BOW,2025,6,4,2025-06-06,1.010272,0.971696,0.996986,0.879623,1.105407,1.442331,24.508713,0.057014
529,20.430000,20.440001,19.969999,20.410000,1300,0.0,0.0,HDL,2025,6,4,2025-06-06,0.998532,1.023160,1.039206,0.969366,0.869992,1.007155,18.816306,0.051134
767,10.510000,10.510000,10.510000,10.510000,0,0.0,0.0,RFAI,2025,6,4,2025-06-06,1.000000,1.000952,1.002863,1.007960,1.024366,NaN,0.356043,NaN
1039,0.380000,0.386000,0.201000,0.295000,3428900,0.0,0.0,JDZG,2025,6,4,2025-06-06,0.862573,0.565134,0.556604,0.564054,0.393858,0.168571,1.475159,0.083768
1311,1.110000,1.270000,1.100000,1.255000,501000,0.0,0.0,RAY,2025,6,4,2025-06-06,1.081897,0.774691,0.456364,0.459707,0.980469,0.343459,9.314924,0.032041
1584,1.210000,1.320000,1.161000,1.290000,21500,0.0,0.0,BTOC,2025,6,4,2025-06-06,1.084034,1.015748,1.057377,1.084034,0.331620,0.261663,3.101715,0.069853
1859,26.650000,26.650000,25.924999,26.389999,580300,0.0,0.0,ZK,2025,6,4,2025-06-06,0.989130,0.989501,1.003422,1.295533,1.055178,1.028048,45.482328,0.021614
2099,10.560000,10.560000,10.560000,10.560000,1800,0.0,0.0,GPAT,2025,6,4,2025-06-06,1.001898,1.004757,1.007634,1.020684,1.035294,NaN,0.879517,NaN
2375,7.710000,7.780000,7.250000,7.320000,123700,0.0,0.0,PAL,2025,6,4,2025-06-06,0.971467,0.915000,0.954368,0.802632,0.743147,0.484447,9.307776,0.047213
2651,4.820000,4.959000,4.750000,4.810000,50700,0.0,0.0,SVCO,2025,6,4,2025-06-06,1.010504,0.971717,1.054825,0.928571,0.579518,0.280303,6.044463,0.038929


In [8]:
filtered_stocks_df = filtered_stocks_df.dropna()
filtered_stocks_df['growth_252d'].describe()

count    71.000000
mean      1.152898
std       1.406017
min       0.024970
25%       0.293422
50%       0.758065
75%       1.362736
max       8.097413
Name: growth_252d, dtype: float64

In [9]:
subset_df = filtered_stocks_df[['Ticker', 'Date', 'growth_252d', 'Sharpe']]
subset_df = subset_df.dropna()
subset_df.head(75)

,Ticker,Date,growth_252d,Sharpe
259,BOW,2025-06-06,1.442331,0.057014
529,HDL,2025-06-06,1.007155,0.051134
1039,JDZG,2025-06-06,0.168571,0.083768
1311,RAY,2025-06-06,0.343459,0.032041
1584,BTOC,2025-06-06,0.261663,0.069853
1859,ZK,2025-06-06,1.028048,0.021614
2375,PAL,2025-06-06,0.484447,0.047213
2651,SVCO,2025-06-06,0.280303,0.038929
2928,NNE,2025-06-06,4.655224,0.080707
3457,VIK,2025-06-06,1.526118,0.040265


In [10]:
subset_df['growth_252d'].describe()

count    71.000000
mean      1.152898
std       1.406017
min       0.024970
25%       0.293422
50%       0.758065
75%       1.362736
max       8.097413
Name: growth_252d, dtype: float64

In [11]:
subset_df['Sharpe'].describe()

count    71.000000
mean      0.288285
std       0.519028
min      -0.079677
25%       0.041215
50%       0.083768
75%       0.311507
max       2.835668
Name: Sharpe, dtype: float64

In [12]:
print("The median Sharpe rate is 0.08 for 71 stocks")

The median Sharpe rate is 0.08 for 71 stocks


In [13]:
sorted_252d_growth = subset_df.sort_values(by='growth_252d', ascending=False)
sorted_252d_growth.head(10)

,Ticker,Date,growth_252d,Sharpe
20287,JL,2025-06-06,8.097413,0.566222
23089,ROMA,2025-06-06,6.156406,0.486840
14147,UMAC,2025-06-06,4.966533,0.421306
2928,NNE,2025-06-06,4.655224,0.080707
4600,RBRK,2025-06-06,3.184065,0.019051
16516,AHR,2025-06-06,2.483097,0.112576
17851,AS,2025-06-06,2.478203,0.028026
4314,MRX,2025-06-06,2.300384,0.076540
11401,RDDT,2025-06-06,2.225505,0.019543
6916,MTEN,2025-06-06,2.210432,0.053200


In [14]:
sorted_sharpe = subset_df.sort_values(by='Sharpe', ascending=False)
sorted_sharpe.head(10)

,Ticker,Date,growth_252d,Sharpe
10782,BKHA,2025-06-06,1.045881,2.835668
20955,JVSA,2025-06-06,1.071076,2.041531
16821,LEGT,2025-06-06,1.049407,1.940267
10199,IBAC,2025-06-06,1.044611,1.637119
14485,HLXB,2025-06-06,1.061404,1.123493
8096,MNDR,2025-06-06,1.030769,0.974234
13479,DYCQ,2025-06-06,1.059863,0.969321
12023,INTJ,2025-06-06,0.661386,0.744512
20287,JL,2025-06-06,8.097413,0.566222
5756,TRSG,2025-06-06,0.646209,0.518080


In [15]:
print("Only one of the companies is in the top 10 for both the Sharpe ratio and 252 day growth")

Only one of the companies is in the top 10 for both the Sharpe ratio and 252 day growth


In [1]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
import yfinance as yf
import time
tickers = ['BOW', 'HDL', 'RFAI', 'JDZG', 'RAY', 'BTOC', 'ZK', 'GPAT', 'PAL', 'SVCO', 'NNE', 'CCIX', 'VIK', 'ZONE', 'LOAR', 'MRX', 'RBRK', 'NCI', 'MFI', 'YYGH', 'TRSG', 'CDTG', 'CTRI', 'IBTA', 'MTEN', 'TWG', 'ULS', 'PACS', 'MNDR', 'CTNM', 'MAMO', 'ZBAO', 'BOLD', 'MMA', 'UBXG', 'IBAC', 'AUNA', 'BKHA', 'LOBO', 'RDDT', 'ALAB', 'INTJ', 'RYDE', 'LGCL', 'SMXT', 'VHAI', 'DYCQ', 'CHRO', 'UMAC', 'HLXB', 'MGX', 'TBBB', 'TELO', 'KYTX', 'PMNT', 'AHR', 'LEGT', 'ANRO', 'GUTS', 'AS', 'FBLG', 'AVBP', 'BTSG', 'HAO', 'CGON', 'YIBO', 'JL', 'SUGP', 'JVSA', 'KSPI', 'CCTG', 'PSBD', 'SYNX', 'SDHC', 'ROMA']
stocks1_df = pd.DataFrame({'A' : []})

for i, t in enumerate(tickers): 
   # print(i, t)
    hPrices = yf.Ticker(t).history(period = "max", interval = "1d")
    hPrices['Ticker'] = t
    hPrices['Year']= hPrices.index.year
    hPrices['Month'] = hPrices.index.month
    hPrices['Weekday'] = hPrices.index.weekday
    hPrices['Date'] = hPrices.index.date
    # historical returns

    for i in range(1, 13):
        hPrices['future_growth_'+str(i)+'m'] = hPrices['Close'].shift(-21*i) / hPrices['Close']

   # sleep 1 sec between downloads - not to overload the API server
    time.sleep(1)
    
    
    if stocks1_df.empty:
        stocks1_df = hPrices
    else:
        stocks1_df = pd.concat([stocks1_df, hPrices], ignore_index=True)
pd.set_option('display.max_rows', None)
stocks1_df.head(75)    

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Year,Month,...,future_growth_3m,future_growth_4m,future_growth_5m,future_growth_6m,future_growth_7m,future_growth_8m,future_growth_9m,future_growth_10m,future_growth_11m,future_growth_12m
0,23.000000,24.270000,22.139999,23.799999,3335800,0.0,0.0,BOW,2024,5,...,1.301681,1.167647,1.239076,1.461765,1.443698,1.357983,1.389076,1.722689,1.698319,1.533613
1,24.260000,26.150000,23.980000,25.700001,990500,0.0,0.0,BOW,2024,5,...,1.196498,1.090661,1.148249,1.392996,1.354086,1.280934,1.276654,1.640078,1.589883,1.445914
2,25.850000,26.879999,25.075001,26.480000,555100,0.0,0.0,BOW,2024,5,...,1.155589,1.048338,1.111027,1.361783,1.327039,1.233761,1.270015,1.535121,1.517749,1.408610
3,26.440001,26.490000,25.500999,26.290001,302700,0.0,0.0,BOW,2024,5,...,1.149106,1.053252,1.139977,1.378851,1.311525,1.248764,1.315709,1.559528,1.521491,1.430582
4,27.209999,27.209999,25.500000,26.139999,200900,0.0,0.0,BOW,2024,5,...,1.198164,1.071538,1.149579,1.383321,1.321729,1.236802,1.303367,1.595639,1.584545,1.432670
5,26.490000,26.990000,25.100000,26.799999,198800,0.0,0.0,BOW,2024,5,...,1.155970,1.092911,1.097015,1.374627,1.325373,1.213060,1.282836,1.549627,1.541045,1.372761
6,27.000000,27.490000,26.360001,26.639999,283500,0.0,0.0,BOW,2024,6,...,1.152027,1.068694,1.092718,1.366742,1.289039,1.247748,1.265766,1.436186,1.538288,1.352102
7,26.129999,26.900000,25.200001,25.230000,169800,0.0,0.0,BOW,2024,6,...,1.248117,1.127230,1.164487,1.419342,1.341657,1.326199,1.327388,1.481570,1.531510,1.442331
8,25.160000,25.900000,24.370001,25.360001,348400,0.0,0.0,BOW,2024,6,...,1.225946,1.148265,1.170347,1.447161,1.272476,1.332413,1.310726,1.480284,1.541798,1.392350
9,25.400000,26.200001,25.170000,25.420000,100000,0.0,0.0,BOW,2024,6,...,1.225020,1.110543,1.172305,1.487018,1.266719,1.312352,1.333989,1.511802,1.554681,1.373721


In [2]:
pd.to_datetime(stocks1_df['Date'], format='%Y-%m-%d')
print(stocks1_df.columns)
min_dates = stocks1_df.groupby('Ticker')['Date'].min().reset_index()

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits',
       'Ticker', 'Year', 'Month', 'Weekday', 'Date', 'future_growth_1m',
       'future_growth_2m', 'future_growth_3m', 'future_growth_4m',
       'future_growth_5m', 'future_growth_6m', 'future_growth_7m',
       'future_growth_8m', 'future_growth_9m', 'future_growth_10m',
       'future_growth_11m', 'future_growth_12m'],
      dtype='object')


In [3]:
min_dates.head(77)

,Ticker,Date
0,AHR,2024-02-07
1,ALAB,2024-03-20
2,ANRO,2024-02-02
3,AS,2024-02-01
4,AUNA,2024-03-22
5,AVBP,2024-01-26
6,BKHA,2024-05-13
7,BOLD,2024-03-28
8,BOW,2024-05-23
9,BTOC,2024-05-14


In [4]:
stocks2_df = pd.merge(min_dates, stocks1_df, on=['Ticker', 'Date'], how='inner')
stocks2_df = stocks2_df.dropna()
stocks2_df.head(78)

,Ticker,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Year,...,future_growth_3m,future_growth_4m,future_growth_5m,future_growth_6m,future_growth_7m,future_growth_8m,future_growth_9m,future_growth_10m,future_growth_11m,future_growth_12m
0,AHR,2024-02-07,12.085785,12.471402,11.878869,12.433781,12732800,0.0,0.0,2024,...,1.060660,1.125315,1.214961,1.308118,1.753552,1.963886,1.988395,2.257203,2.259751,2.273312
1,ALAB,2024-03-20,52.560001,63.500000,50.610001,62.029999,16843300,0.0,0.0,2024,...,1.018217,0.853780,0.631307,0.759794,1.078027,1.457037,1.974367,1.982105,1.381912,1.189424
2,ANRO,2024-02-02,22.000000,23.270000,20.000000,20.700001,2386300,0.0,0.0,2024,...,0.686957,0.546377,0.563768,0.438164,0.647343,0.492270,0.192271,0.203865,0.222705,0.202899
3,AS,2024-02-01,13.400000,13.800000,13.100000,13.400000,18656400,0.0,0.0,2024,...,1.080597,1.143284,0.935075,0.802239,0.999254,1.236567,1.332090,1.950000,2.152985,2.362687
4,AUNA,2024-03-22,9.510000,10.320000,9.300000,9.600000,9046900,0.0,0.0,2024,...,0.801042,0.919792,0.782292,0.767708,0.723958,0.733333,0.706250,0.857292,0.850000,0.752083
5,AVBP,2024-01-26,24.000000,25.950001,20.000000,20.000000,1992600,0.0,0.0,2024,...,0.772000,0.948500,0.870000,1.084000,1.386000,1.175500,1.584000,1.375000,1.379000,1.451000
6,BKHA,2024-05-13,10.200000,10.200000,10.130000,10.130000,1400,0.0,0.0,2024,...,1.004936,1.006910,1.011846,1.022705,1.021718,1.026653,1.029615,1.037512,1.042448,1.045410
7,BOLD,2024-03-28,14.250000,15.240000,14.100000,14.250000,1754100,0.0,0.0,2024,...,0.271579,0.267368,0.251930,0.237895,0.210526,0.186667,0.178947,0.171228,0.121053,0.101754
8,BOW,2024-05-23,23.000000,24.270000,22.139999,23.799999,3335800,0.0,0.0,2024,...,1.301681,1.167647,1.239076,1.461765,1.443698,1.357983,1.389076,1.722689,1.698319,1.533613
9,BTOC,2024-05-14,5.000000,6.260000,4.139000,4.620000,1323900,0.0,0.0,2024,...,1.038961,0.909091,0.995671,1.238095,1.158009,0.922078,0.735931,0.222944,0.244589,0.341991


In [5]:
for i in range(1, 13):
    print(stocks2_df['future_growth_'+str(i)+'m'].describe())

count    71.000000
mean      0.931108
std       0.350683
min       0.098947
25%       0.778984
50%       0.966757
75%       1.063656
max       2.646505
Name: future_growth_1m, dtype: float64
count    71.000000
mean      0.944331
std       0.587887
min       0.073800
25%       0.685815
50%       1.000000
75%       1.170336
max       4.874759
Name: future_growth_2m, dtype: float64
count    71.000000
mean      0.832425
std       0.416743
min       0.060947
25%       0.511212
50%       0.840000
75%       1.078136
max       2.040000
Name: future_growth_3m, dtype: float64
count    71.000000
mean      0.823113
std       0.408509
min       0.045368
25%       0.517233
50%       0.880011
75%       1.155465
max       1.605000
Name: future_growth_4m, dtype: float64
count    71.000000
mean      0.802068
std       0.496661
min       0.054109
25%       0.448403
50%       0.811005
75%       1.017884
max       3.213873
Name: future_growth_5m, dtype: float64
count    71.000000
mean      0.867521
std    

In [6]:
print("After 2 months the mean future growth is 0.94 which is the max")

After 2 months the mean future growth is 0.94 which is the max


In [1]:
import gdown
import pandas as pd

file_id = "1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-"
gdown.download(f"https://drive.google.com/uc?id={file_id}", "data.parquet", quiet=False)
df = pd.read_parquet("data.parquet", engine="pyarrow")

Downloading...
From (original): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-
From (redirected): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-&confirm=t&uuid=f544ef04-76a6-4f9f-8f22-f6d3b2d00162
To: /home/julie/market_analytics/data.parquet
100%|████████████████████████████████████████| 130M/130M [00:06<00:00, 18.6MB/s]


In [2]:
rsi_threshold = 25
selected_df = df[
    (df['rsi'] < rsi_threshold) &
    (df['Date'] >= '2000-01-01') &
    (df['Date'] <= '2025-06-01')
]

In [8]:
pd.set_option('display.max_columns', None)

selected_df.head()

,Open,High,Low,Close_x,Volume,Dividends,Stock Splits,Ticker,Year,Month,Weekday,Date,growth_1d,growth_3d,growth_7d,growth_30d,growth_90d,growth_365d,growth_future_30d,SMA10,SMA20,growing_moving_average,high_minus_low_relative,volatility,is_positive_growth_30d_future,ticker_type,index_x,adx,adxr,apo,aroon_1,aroon_2,aroonosc,bop,cci,cmo,dx,macd,macdsignal,macdhist,macd_ext,macdsignal_ext,macdhist_ext,macd_fix,macdsignal_fix,macdhist_fix,mfi,minus_di,mom,plus_di,dm,ppo,roc,rocp,rocr,rocr100,rsi,slowk,slowd,fastk,fastd,fastk_rsi,fastd_rsi,trix,ultosc,willr,index_y,ad,adosc,obv,atr,natr,ht_dcperiod,ht_dcphase,ht_phasor_inphase,ht_phasor_quadrature,ht_sine_sine,ht_sine_leadsine,ht_trendmod,avgprice,medprice,typprice,wclprice,index,cdl2crows,cdl3blackrows,cdl3inside,cdl3linestrike,cdl3outside,cdl3starsinsouth,cdl3whitesoldiers,cdlabandonedbaby,cdladvancedblock,cdlbelthold,cdlbreakaway,cdlclosingmarubozu,cdlconcealbabyswall,cdlcounterattack,cdldarkcloudcover,cdldoji,cdldojistar,cdldragonflydoji,cdlengulfing,cdleveningdojistar,cdleveningstar,cdlgapsidesidewhite,cdlgravestonedoji,cdlhammer,cdlhangingman,cdlharami,cdlharamicross,cdlhighwave,cdlhikkake,cdlhikkakemod,cdlhomingpigeon,cdlidentical3crows,cdlinneck,cdlinvertedhammer,cdlkicking,cdlkickingbylength,cdlladderbottom,cdllongleggeddoji,cdllongline,cdlmarubozu,cdlmatchinglow,cdlmathold,cdlmorningdojistar,cdlmorningstar,cdlonneck,cdlpiercing,cdlrickshawman,cdlrisefall3methods,cdlseparatinglines,cdlshootingstar,cdlshortline,cdlspinningtop,cdlstalledpattern,cdlsticksandwich,cdltakuru,cdltasukigap,cdlthrusting,cdltristar,cdlunique3river,cdlupsidegap2crows,cdlxsidegap3methods,growth_dax_1d,growth_dax_3d,growth_dax_7d,growth_dax_30d,growth_dax_90d,growth_dax_365d,growth_snp500_1d,growth_snp500_3d,growth_snp500_7d,growth_snp500_30d,growth_snp500_90d,growth_snp500_365d,growth_dji_1d,growth_dji_3d,growth_dji_7d,growth_dji_30d,growth_dji_90d,growth_dji_365d,growth_epi_1d,growth_epi_3d,growth_epi_7d,growth_epi_30d,growth_epi_90d,growth_epi_365d,Quarter,gdppot_us_yoy,gdppot_us_qoq,cpi_core_yoy,cpi_core_mom,FEDFUNDS,DGS1,DGS5,DGS10,Close_y,growth_gold_1d,growth_gold_3d,growth_gold_7d,growth_gold_30d,growth_gold_90d,growth_gold_365d,growth_wti_oil_1d,growth_wti_oil_3d,growth_wti_oil_7d,growth_wti_oil_30d,growth_wti_oil_90d,growth_wti_oil_365d,growth_brent_oil_1d,growth_brent_oil_3d,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
3668,20.056772,20.114241,19.405453,19.673643,99915200.0,0.0,0.0,MSFT,2000,2000-09-01,4,2000-09-15,0.975309,0.942202,0.924393,0.913701,0.946544,0.678784,0.985394,20.974362,21.328757,0,0.036027,10.048792,0,US,3668,12.278538,12.354158,-0.443053,100.000000,14.285714,-85.714286,-0.540541,-226.933219,-50.444133,38.817783,-0.439947,-0.293928,-0.146019,-0.443053,-0.258039,-0.185015,-0.417009,-0.279346,-0.137663,40.992443,2.533460,-1.724079,13.788887,1.116591,-2.061408,-8.057302,-0.080573,0.919427,91.942698,24.777933,12.643311,13.912738,13.725444,12.643311,0.0,3.546340e+00,-0.114896,29.048809,-90.728477,3668,1.285246e+10,-9.334272e+07,1.370745e+10,0.578411,2.940032,19.237347,-21.778977,-0.144480,-0.300240,-0.371027,0.394279,1,19.812527,19.759847,19.731112,19.716745,3668,0,0,0,0,-100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.993054,0.980912,0.954524,0.997570,0.962893,1.339568,0.989830,0.989082,0.982282,1.009122,1.038006,1.090649,0.985527,0.972739,0.966082,1.020587,1.037037,1.071516,NaN,NaN,NaN,NaN,NaN,NaN,2000-07-01,0.042748,0.009916,0.025309,0.002199,6.52,6.10,5.93,5.84,18.520000,0.999633,0.997801,0.993071,NaN,NaN,NaN,1.051320,1.048246,1.025751,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3669,19.692798,19.807736,19.060636,19.309669,69037800.0,0.0,0.0,MSFT,2000,2000-09-01,0,2000-09-18,0.981499,0.923077,0.899197,0.911392,0.951841,0.668435,0.972222,20.754

In [3]:
net_income = 1000 * (selected_df['growth_future_30d'] - 1).sum()

In [4]:
net_income

np.float64(24295.523125248386)

In [9]:
"The net income over 25 years for holding oversold stocks for 30 days is $24295"

'The net income over 25 years for holding oversold stocks for 30 days is $24295'